### PDF파일 하나 업로드하고 대화하기

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
import streamlit as st
import time
import base64
import uuid
import tempfile
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader

from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage, SystemMessage

In [3]:
chat = ChatUpstage(upstage_api_key=os.getenv("UPSTAGE_API_KEY"))

messages = [
  SystemMessage(content="You are a helpful assistant."),
  HumanMessage(content="Hi, how are you?")
]
response = chat.invoke(messages)
print(response.content)

Good evening! I'm doing well, thank you for asking. How about you?


### PDF 파일 업로드

In [4]:
# from langchain.document_loaders import PyPDFLoader

from langchain_community.document_loaders import PyPDFLoader

In [5]:
loader = PyPDFLoader("../pdfs/EU-탄소중립산업법-주요-내용.pdf")


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
    )

pages = loader.load_and_split(text_splitter)

pages[:5]

[Document(page_content='1. 도입 배경\n2021.12.8, EU집행위는 제3국의 경제적 위협 으로부터 유럽연합 (EU) 및 회원국을 보호하기 위해 통상위협대응 규정 (안)을 \n발표했습니다 . EU집행위는 EU가 제3국의 경제적 위협에 신속히 대응하고 필요시 최후의 수단으로 대응조치\n(countermeasure) 를 취하여 EU와 회원국의 경제적 이익을 보호 하기 위해 본 규정 (안)을 준비했습니다 . (제1조) \n본 규정 (안)은 지정학적 긴장이 고조되며 , 통상이 점점 더 무기화되어 가고 있는 시점에 EU가 통상 정책을 강화하고 , 제3국의 \n경제적 위협에 효과적으로 대응하기 위한 해결방안으로 제시한 것입니다 . 2018.3 미국이 철강 232 조 관세를 부과함에 따라 \nEU는 직접적인 피해를 입었습니다 . 또한 , 2019. 12 세계무역기구 TO) 상소기구가 마비되어 WTO 분쟁해결의 효율성이 약 없\n게 되어 , EU는 경제적 위협에 더욱 취약해졌습니다 .\n본 규정 (안)의 도입은 2019.12 집행위가 통상권한집행 규정 (Trade Enforcement Regulation) 을 개정하기 위한 입법안을 \n제시한 후 처음 논의가 되었습니다 . 당시 유럽의회는 경제적 위협에 대한 우려를 제기했지만 , 개정된 통상권한집행 규정은 EU가 \n경제적 위협에 대응할 수 있는 방법을 규정하지 않았습니다 . 그 후속조치로 집행위 , 의회 , 이사회는 2021 년 말 까지 집행위가 경\n제적 위협에 대응하기 위한 별도의 입법안을 마련하겠다는 것을 명시한 공동 선언문을 채택했습니다 .\n그 결과 이번에 발표된 통상위협대응 규정 (안)은 제3국의 경제적 위협에 대응하기 위해 EU가 취할 조치를 2단계로 명시하고 있\n습니다 . 첫째 , 집행위는 제3국의 경제적 위협을 분석하여 , 그 위협의 유무를 공식적으로 결정한 후, 경제적 위협이 중단될 수 있\n도록 제3국과 협의를 합니다 . 둘째 , 제3국과의 협의가 경제적 위협을 중단하는 결과를 

In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

from langchain.vectorstores import Chroma

directory = "./pdf_db"       # 경로 지정 -> 크로마 함수에서 폴더 만들어 줄것임


vectorstore = Chroma.from_documents(
                        pages, 
                        UpstageEmbeddings(model="solar-embedding-1-large"),
                        persist_directory=directory # persists the vectors to the file system
                        )


vectorstore.persist()

c:\Users\User\anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [8]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os 
from langchain.chains import RetrievalQA

# 검색기 설정 
retriever = vectorstore.as_retriever(
                                search_type="similarity", # Cosine Similarity
                                search_kwargs={"k": 3,} 
                                
)


In [9]:
# QA 체인 설정
qa = RetrievalQA.from_llm(llm=chat, retriever=retriever, return_source_documents=True)

In [10]:
query = '탄소중립 전략 프로젝트에 대해서 알려줘.'
result = qa(query)          # 질문 -> qa -> answer

print(result["result"])     # 대답 출력 

c:\Users\User\anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


탄소중립 전략 프로젝트는 EU에서 탄소중립을 달성하기 위해 중요한 역할을 하는 프로젝트입니다. 이 프로젝트는 EU 내에서 8개의 핵심 탄소중립 기술의 제조 역량을 강화하고, EU 에너지 시스템의 회복력을 향상시키는 데 기여합니다. 또한, EU 내 탄소 포집, 활용 및 저장 (CCUS) 프로젝트도 탄소중립 전략 프로젝트로 선정됩니다. EU는 2030년까지 연간 5천만 톤의 CO2를 저장할 수 있는 능력을 갖추는 것을 목표로 하고 있습니다. 탄소 포집 프로젝트의 경우, 각 회원국은 잠재적인 저장소, 탄소 포집 프로젝트 현황 등의 데이터를 수집하여 집행위원회에 보고해야 합니다. 천연가스 및 원유 채굴권을 가진 기업들은 이산화탄소 주입 및 저장을 위한 목표를 설정하고, 탄소 포집 프로젝트 개발 등의 세부 계획을 제출해야 합니다. 이러한 전략 프로젝트는 부지 조성, 설계, 인프라 구축 등의 허가 절차를 우선적으로 처리하며, 행정 절차, 민원 등을 지원합니다.


### 메세지 기억하고 대답 근거 출력하기

In [11]:
# 1) 챗봇에 '기억'을 입히기 위한 첫번째 단계 

# 이전의 메시지들과 최신 사용자 질문을 분석해, 문맥에 대한 정보가 없이 혼자서만 봤을때 이해할 수 있도록 질문을 다시 구성함
# 즉 새로 들어온 그 질문 자체에만 집중할 수 있도록 다시 재편성
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """이전 대화 내용과 최신 사용자 질문이 있을 때, 이 질문이 이전 대화 내용과 관련이 있을 수 있습니다. 
이런 경우, 대화 내용을 알 필요 없이 독립적으로 이해할 수 있는 질문으로 바꾸세요. 
질문에 답할 필요는 없고, 필요하다면 그저 다시 구성하거나 그대로 두세요."""

# MessagesPlaceholder: 'chat_history' 입력 키를 사용하여 이전 메세지 기록들을 프롬프트에 포함시킴.
# 즉 프롬프트, 메세지 기록 (문맥 정보), 사용자의 질문으로 프롬프트가 구성됨. 
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# 이를 토대로 메세지 기록을 기억하는 retriever를 생성합니다.
history_aware_retriever = create_history_aware_retriever(
    chat, retriever, contextualize_q_prompt
)


In [12]:
# 2) 두번째 단계로, 방금 전 생성한 체인을 사용하여 문서를 불러올 수 있는 retriever 체인을 생성합니다.
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """질문-답변 업무를 돕는 보조원입니다. 
질문에 답하기 위해 검색된 내용을 사용하세요. 
답을 모르면 모른다고 말하세요. 
답변은 세 문장 이내로 간결하게 유지하세요.

## 답변 예시
📍답변 내용: 
📍증거: 

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(chat, qa_prompt)

# 결과값은 input, chat_history, context, answer 포함함.
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [13]:
history_messages = []

In [14]:
prompt = input()
# 1. EU 탄소중립산업법에서 탄소중립플랫폼(Net-Zero Europe Platform) 설립 부분에 대해 설명해줘.
# 2. EU 경제안보전략 입법 현황에서 수출통제 백서에 대해 이사회의 입장을 알려줘.

history_messages.append(prompt)

result = rag_chain.invoke({"input": prompt, "chat_history": history_messages})


sentences = result["answer"].split('.')

for i in range(len(sentences)):
    print(sentences[i])
    
print('---'*10)


for con in result["context"]:
    print(con)


EU 탄소중립산업법은 탄소중립 산업으로의 전환을 지원하기 위한 입법 패키지입니다
 이 법의 중요한 요소 중 하나는 탄소중립플랫폼(Net-Zero Europe Platform)의 설립입니다


탄소중립플랫폼은 탄소중립 산업으로의 전환을 지원하기 위한 중앙 집중식 협의 및 협력 메커니즘으로 기능합니다
 이 플랫폼은 EU 회원국, 지역 및 지방 정부, 산업 대표, 노동조합, 시민 사회 단체 등 다양한 이해관계자들을 모아 탄소중립 전환에 대한 조정과 협력을 촉진합니다


이 플랫폼은 탄소중립 전환을 위한 정책 및 전략에 대한 의견을 제공하고, 모범 사례를 공유하며, 전환 과정에서 발생하는 문제점에 대한 해결책을 모색합니다
 또한, 정책 및 규제에 대한 권고사항을 제시하고, 탄소중립 전환을 위한 자금 및 지원 프로그램에 대한 정보를 제공합니다


탄소중립플랫폼의 설립은 EU가 탄소중립 목표를 달성하기 위해 필요한 협력과 조정을 강화하기 위한 중요한 단계입니다
 이를 통해 다양한 이해관계자들이 협력하고 지식을 공유함으로써 탄소중립 산업으로의 전환을 더욱 효과적으로 추진할 수 있을 것으로 기대됩니다

------------------------------
page_content='* 참고 : <탄소중립산업법 조항 개요>\n*참고:한국무역협회 , ‘EU 탄소중립산업법 주요 내용과 시사점 ’\n한국무역협회 브뤼셀지부 \n/ KBA Europe 사무국Avenue Louise 66, 1050 Brussels, Belgium\nTel   +32 2 639 0990  / Homepage   kba-europe.com  \n Email   kba@kba-europe.com\n   * EU Trade Brief 및 KBA 뉴스레터 구독 문의는 kba@kba-europe.com 으로 메일 주시기 바랍니다 .구분 항목\n1장. 주제, 범위, 정의1조 주제\n2조 범위\n3조 정의\n1절. 간소화 및 허가 과정\n2장. 탄소중립 기술 \n제조를 위한 여건 \n마련4조 행정창구 일원화\n5조 

In [15]:
loader = PyPDFLoader("../pdfs/EU경제안보전략이니셔티브-입법-현황.pdf")

In [16]:
pages2 = loader.load_and_split(text_splitter)

In [17]:
vectorstore.get()

{'ids': ['029ef2b9-eadf-4309-8646-c284adc24c30',
  '09e95f4b-61c1-4ee9-bf58-3771f6b3a88b',
  '0b58abbd-a0b8-49ef-b565-edae20e1b3b5',
  '17acd51a-45af-483d-9dc8-3fb8a5ae9f6d',
  '1d4c65c7-825d-4660-8cf7-c585dad30fdf',
  '250563e6-bf49-4695-a745-cc1243677bfb',
  '273f9fff-2cf7-4979-aae5-c77b26cc3f3d',
  '27a08592-0a85-4950-b1a9-10d2235b1440',
  '27a29fb4-2439-4b2e-897d-d16e3caa0b99',
  '28b326b5-f0a0-4a86-aab6-b3dcecc240e1',
  '28bd6d04-d8db-41b3-9141-8d0d11c3eb71',
  '2982baf9-46fb-4059-9597-8e17fca66858',
  '30092516-c3fb-44f5-b372-60bf1f98404f',
  '35b601c8-4c2a-4950-bc2a-6fea7a4fc99e',
  '36046d91-ddad-47f0-ab8f-d4fb4c9f06ca',
  '43f70152-894b-477c-abd3-a37dac5f12a1',
  '473aba31-b895-4aaa-b31d-ddef42e0c965',
  '4753904d-d4cb-4aff-b3e6-2d6c4358e17c',
  '52487f46-98ad-4fe4-82b2-4a4f0a386b1c',
  '66658181-5679-4205-b171-8020e6bff879',
  '68905eb5-a054-45c5-977d-8c442065245b',
  '6dfa7fd3-956b-4a21-8d47-1acfbbbd167a',
  '70e00cfd-b3a6-419a-b629-926fb0be400b',
  '73c09bbc-b5b4-45d1-8f7b-

In [18]:
vectorstore.add_documents(pages2)

['712a7379-c570-402f-a6fc-738a2393155e',
 'a20b1d54-a3d5-40eb-aeef-3d6fa6bcb2fa',
 '90742a79-520b-47e2-a133-112ea4a948e1',
 'c53e1aaa-18e6-40b8-b41a-7c2cf3c88df9',
 '102b8a7b-371e-4b82-9830-aacae6511a31']

In [19]:
vectorstore.get()

{'ids': ['029ef2b9-eadf-4309-8646-c284adc24c30',
  '09e95f4b-61c1-4ee9-bf58-3771f6b3a88b',
  '0b58abbd-a0b8-49ef-b565-edae20e1b3b5',
  '102b8a7b-371e-4b82-9830-aacae6511a31',
  '17acd51a-45af-483d-9dc8-3fb8a5ae9f6d',
  '1d4c65c7-825d-4660-8cf7-c585dad30fdf',
  '250563e6-bf49-4695-a745-cc1243677bfb',
  '273f9fff-2cf7-4979-aae5-c77b26cc3f3d',
  '27a08592-0a85-4950-b1a9-10d2235b1440',
  '27a29fb4-2439-4b2e-897d-d16e3caa0b99',
  '28b326b5-f0a0-4a86-aab6-b3dcecc240e1',
  '28bd6d04-d8db-41b3-9141-8d0d11c3eb71',
  '2982baf9-46fb-4059-9597-8e17fca66858',
  '30092516-c3fb-44f5-b372-60bf1f98404f',
  '35b601c8-4c2a-4950-bc2a-6fea7a4fc99e',
  '36046d91-ddad-47f0-ab8f-d4fb4c9f06ca',
  '43f70152-894b-477c-abd3-a37dac5f12a1',
  '473aba31-b895-4aaa-b31d-ddef42e0c965',
  '4753904d-d4cb-4aff-b3e6-2d6c4358e17c',
  '52487f46-98ad-4fe4-82b2-4a4f0a386b1c',
  '66658181-5679-4205-b171-8020e6bff879',
  '68905eb5-a054-45c5-977d-8c442065245b',
  '6dfa7fd3-956b-4a21-8d47-1acfbbbd167a',
  '70e00cfd-b3a6-419a-b629-